<a href="https://colab.research.google.com/github/IlyaProrok/task_2/blob/master/task_2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install nltk==3.5b1

     |████████████████████████████████| 92kB 5.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 1.4MB 7.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.1.1)
     |████████████████████████████████| 1.4 MB 6.7 MB/s 
  Created wheel for nltk: filename=nltk-3.5b1-py3-none-any.whl size=1434694 sha256=3a62905cd70aa8b9bcbc42ecbe64cb97531227d42fa4a3b814e505d0548b1a01
  Stored in directory: /root/.cache/pip/wheels/d3/86/25/c46dd9a7eced213ff414cc8266b44bb26e61f3f787d36d8dfc
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
import lyricsgenius
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
import random
from nltk.lm.preprocessing import flatten
from nltk.lm import Vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
genius = lyricsgenius.Genius("iihP9u6cVKE79Xtu99OH6rVZlMUzIMeOvIVs8BDQNDeVmZhXJXEaI28E6QOc4BdB")
genius.remove_section_headers = True
genius.skip_non_songs = False 
genius.excluded_terms = ["(Remix)", "(Live)"]

In [0]:
Oxxxymiron = genius.search_artist("Oxxxymiron", sort="title")

In [0]:
SlavaKPSS = genius.search_artist("Слава КПСС", sort="title")

In [0]:
NoizeMC = genius.search_artist("Noize MC", sort="title")

In [0]:
FACE = genius.search_artist("FACE", sort="title")

In [0]:
# saving the lyrics of all songs of Oxxxymiron
Oxxxymiron.save_lyrics(extension='txt')
SlavaKPSS.save_lyrics(extension='txt')
NoizeMC.save_lyrics(extension='txt')
FACE.save_lyrics(extension='txt')

Wrote `Lyrics_Oxxxymiron.txt`
Wrote `Lyrics_СлаваКПССSlavaKPSS.txt`
Wrote `Lyrics_NoizeMC.txt`


In [0]:
Oxy_lyrics = open("Lyrics_Oxxxymiron.txt","r")
Slava_lyrics = open("Lyrics_СлаваКПССSlavaKPSS.txt","r")
Noize_lyrics = open("Lyrics_NoizeMC.txt","r")
# FACE_lyrics = open("Lyrics_FACE.txt","r")

In [0]:
# tokenizing the lyrics, probably the tokenization is raw and there is anything I missed?
Oxy_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Oxy_lyrics]
Slava_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Slava_lyrics]
Noize_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Noize_lyrics]
FACE_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  # for sent in FACE_lyrics]  

In [0]:
print(len(Oxy_tokenized),len(Slava_tokenized),len(Noize_tokenized),len(FACE_tokenized))

5280 13440 16633 5487


In [0]:
all_lyrics = []
all_lyrics.extend(Oxy_tokenized)
all_lyrics.extend(Slava_tokenized)
all_lyrics.extend(Noize_tokenized)
all_lyrics.extend(FACE_tokenized)
print(len(all_lyrics))
print(len(Oxy_tokenized)+len(Slava_tokenized)+len(Noize_tokenized)+len(FACE_tokenized))

30164
30164


In [0]:
# Firstly, I was thinking, that adding \n symbol is a good idea to let the model know, where to start the next line,
# but then i neglected the idea
# for strokes in tokenized_text:
#   strokes.append('\n')

In [0]:
# def a function to generate rap text given a model
def rap_gen(model, num_lines = 20, num_words_in_line = 20, random_seed = 0):
  """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
  detokenize = TreebankWordDetokenizer().detokenize
  for i in range(num_lines):
    content = []
    for token in model.generate(num_words_in_line, random_seed=random_seed+i):
      if token == '<s>':
        continue
      if token == '</s>':
        break
      content.append(token)
    content = detokenize(content)
    if bool(content):
      i = 0
      while not content[i].isalpha():
          i += 1
          if i > len(content)-1:
            break
      print(content[i:].capitalize())



In [0]:
train, test = train_test_split(all_lyrics, test_size=0.3, random_state=11)

Unigram models

In [0]:
model1_MLE = MLE(1)
train_data1_1, padded_sents1_1 = padded_everygram_pipeline(1, train)
model1_MLE.fit(train_data1_1, padded_sents1_1)

In [0]:
rap_gen(model1_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Коли оркестром отстой?! ж во сначала по не
Вверх плавниками пират твоего а быть a бы проблем 1
Поездку он что владимир вани проблема основанная высокой писал
Этом! проезжали it я"туссин петушина телами “
Дядька корни и к и орфея ваще ночью! вынул
Меня слушая это всем ртом пойду от ,) за
От дверью барабаны кто курт копов как весь важных
Пенсии свободу меня лягу зале — всем and ведь ведь
Услышав планеты рублём упакован ведут ну упрямо тот надеюсь start
Oh по ночью конч брендом сейчас смена мажет лоли в


In [0]:
model1_Lapl = Laplace(1)
train_data1_2, padded_sents1_2 = padded_everygram_pipeline(1, train)
model1_Lapl.fit(train_data1_2, padded_sents1_2)

In [0]:
rap_gen(model1_Lapl, num_lines=10,num_words_in_line=10,random_seed = 12)

In [0]:
model1_KNI = KneserNeyInterpolated(1)
train_data1_3, padded_sents1_3 = padded_everygram_pipeline(1, train)
model1_KNI.fit(train_data1_3, padded_sents1_3)

In [0]:
rap_gen(model1_KNI, num_lines=10,num_words_in_line=10,random_seed = 12)

Everygram models (until bigram)

In [0]:
model2_MLE = MLE(2)
train_data2_1, padded_sents2_1 = padded_everygram_pipeline(2, train)
model2_MLE.fit(train_data2_1, padded_sents2_1)

In [0]:
rap_gen(model2_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Да вот так, bang, зато построишь дом
По радио и гимном встречу похмельное утро он был
Ипотечными котлами
Эти 3 ночи, я, что с того чтобы

За такую хуйню: ваш трэк, будто
Наш сигнал не ныл
Тупорылый текст писал эсэмэски


In [0]:
model2_Lapl = Laplace(2)
train_data2_2, padded_sents2_2 = padded_everygram_pipeline(2, train)
model2_Lapl.fit(train_data2_2, padded_sents2_2)

In [0]:
rap_gen(model2_Lapl, num_lines=10, num_words_in_line=10,random_seed = 12)

In [0]:
model2_KNI = KneserNeyInterpolated(2)
train_data2_3, padded_sents2_3 = padded_everygram_pipeline(2, train)
model2_KNI.fit(train_data2_3, padded_sents2_3)

In [0]:
rap_gen(model2_KNI, num_lines=10, num_words_in_line=10,random_seed = 12)

Everygram models (until trigram)

In [0]:
model3_MLE = MLE(3)
train_data3_1, padded_sents3_1 = padded_everygram_pipeline(3, train)
model3_MLE.fit(train_data3_1, padded_sents3_1)

In [0]:
rap_gen(model3_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Ока » , 13-го — в шоколаде весь рот
Чём, кроме
Или на парах . (и в интернете
В тапки
Куча копий нашей планеты :
Твоё табло и мега-турбо-супер-хайп


In [0]:
model3_Lapl = Laplace(3)
train_data3_2, padded_sents3_2 = padded_everygram_pipeline(3, train)
model3_Lapl.fit(train_data3_2, padded_sents3_2)

In [0]:
rap_gen(model3_Lapl, num_lines=10, num_words_in_line=10,random_seed = 12)

In [0]:
model3_KNI = KneserNeyInterpolated(3)
train_data3_3, padded_sents3_3 = padded_everygram_pipeline(3, train)
model3_KNI.fit(train_data3_3, padded_sents3_3)

In [0]:
rap_gen(model3_KNI, num_lines=10, num_words_in_line=10,random_seed = 12)

The generated texts are really strange, actually (however, i'm not a rap expert, probably, this is OK :)

Can you also give some advise on a model evaluation with perplexity. Thing I did below, unfortunately, do not give desired result.

In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)
test_data2,_ = padded_everygram_pipeline(2, test)
test_data3,_ = padded_everygram_pipeline(3, test)

In [0]:
test_gen1 = []
test_gen2 = []
test_gen3 = []
for line in test_data1:
  test_gen1.extend(line)
for line in test_data2:
  test_gen2.extend(line)
for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model1_MLE.perplexity(test_gen1)))

Perplexity for unigram MLE model is: inf


In [0]:
print('Perplexity for unigram Laplace model is: {}'.format(model1_Lapl.perplexity(test_gen1)))

In [0]:
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model1_KNI.perplexity(test_gen1)))

In [0]:
print('Perplexity for bigram MLE model is: {}'.format(model2_MLE.perplexity(test_gen2)))

Perplexity for bigram MLE model is: inf


In [0]:
print('Perplexity for bigram Laplace model is: {}'.format(model2_Lapl.perplexity(test_gen2)))

In [0]:
print('Perplexity for bigram KneserNeyInterpolated model is: {}'.format(model2_KNI.perplexity(test_gen2)))

In [0]:
print('Perplexity for trigram MLE model is: {}'.format(model3_MLE.perplexity(test_gen3)))

Perplexity for trigram MLE model is: inf


In [0]:
print('Perplexity for trigram Laplace model is: {}'.format(model3_Lapl.perplexity(test_gen3)))

In [0]:
print('Perplexity for trigram KneserNeyInterpolated model is: {}'.format(model3_KNI.perplexity(test_gen3)))

Everygram models (until trigram)

In [0]:
train_bigram_padded = []
for i in range(len(train)):
  stroke = list(pad_sequence(train[i],
                  pad_left=True, left_pad_symbol="<s>",
                  pad_right=True, right_pad_symbol="</s>",
                  n=2))
  train_bigram_padded.append(stroke)

In [0]:
train_bigram_padded

In [0]:
train_bigram = []
train_bigram = [list(ngrams(train_bigram_padded[i], n=2)) for i in range(len(train_bigram_padded))]

In [0]:
iterat = iter(train_bigram)
iterat

In [0]:
flat_bigram = []
flat_bigram = list(flatten(train_bigram_padded))

In [0]:
vocab_bigram = Vocabulary(flat_bigram,unk_cutoff=2)

In [0]:
model3 = MLE(2)

In [0]:
len(model3.vocab)

0

In [0]:
model3.fit(iterat, vocab_bigram)

In [0]:
len(model3.vocab)

13683

In [0]:
rap_gen(model3)

ValueError: ignored